In [1]:
import numpy as np
from PIL import Image, ImageDraw
import os
import matplotlib.pyplot as plt

In [3]:
def calculate_profiles(image_array):
    profile_x = np.sum(image_array == 0, axis=0)
    profile_y = np.sum(image_array == 0, axis=1)
    return profile_x, profile_y 

def calculate_black_mass(image_array):
    height, width = image_array.shape
    half_height, half_width = height // 2, width // 2

    quarters = [
        image_array[:half_height, :half_width],
        image_array[:half_height, half_width:],
        image_array[half_height:, :half_width],
        image_array[half_height:, half_width:]
    ]

    black_masses = [np.sum(quarter == 0) for quarter in quarters]
    return black_masses

def calculate_normalized_black_mass(image_array):
    height, width = image_array.shape
    quarter_area = (height // 2) * (width // 2)
    
    black_masses = calculate_black_mass(image_array)
    normalized_black_masses = [mass / quarter_area for mass in black_masses]
    
    return normalized_black_masses

def calculate_center_of_gravity(image_array):
    black_pixels = np.argwhere(image_array == 0)
    if len(black_pixels) == 0:
        return (0, 0)
    
    center_of_gravity = np.mean(black_pixels, axis=0)
    return tuple(center_of_gravity)

def calculate_normalized_center_of_gravity(image_array, center_of_gravity):
    height, width = image_array.shape
    normalized_center_of_gravity = (
        center_of_gravity[0] / height,
        center_of_gravity[1] / width
    )
    return normalized_center_of_gravity

def calculate_moments_of_inertia(image_array, center_of_gravity):
    height, width = image_array.shape
    y_center, x_center = center_of_gravity

    y_indices, x_indices = np.indices((height, width))
    y_indices = y_indices - y_center
    x_indices = x_indices - x_center

    horizontal_moment = np.sum((image_array == 0) * (x_indices ** 2))
    vertical_moment = np.sum((image_array == 0) * (y_indices ** 2))

    return horizontal_moment, vertical_moment

def calculate_normalized_moments_of_inertia(image_array, moments_of_inertia):
    height, width = image_array.shape
    area = height * width

    normalized_horizontal_moment = moments_of_inertia[0] / area
    normalized_vertical_moment = moments_of_inertia[1] / area

    return normalized_horizontal_moment, normalized_vertical_moment

def calculate_features(image_array):
    black_masses = calculate_black_mass(image_array)
    normalized_black_masses = calculate_normalized_black_mass(image_array)
    center_of_gravity = calculate_center_of_gravity(image_array)
    normalized_center_of_gravity = calculate_normalized_center_of_gravity(image_array, center_of_gravity)
    moments_of_inertia = calculate_moments_of_inertia(image_array, center_of_gravity)
    normalized_moments_of_inertia = calculate_normalized_moments_of_inertia(image_array, moments_of_inertia)

    features = normalized_black_masses + list(normalized_center_of_gravity) + list(normalized_moments_of_inertia)
    return features

In [4]:
def euclidean_distance(features1, features2):
    return np.sqrt(np.sum((np.array(features1) - np.array(features2)) ** 2))

def similarity_measure(features1, features2):
    distance = euclidean_distance(features1, features2)
    return 1 / (1 + distance)